In [ ]:
# exit()
# uv pip install ipykernel

In [1]:
# Jupyterlab에서 django shell을 실행시키기 위한 설정 (shell: project 모듈들을 테스트 할 수있는 파이썬 shell)

import os
import django

os.environ['DJANGO_SETTINGS_MODULE'] = "config.settings" # config/settings.py
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

django.setup()

In [ ]:
from polls.models import Question, Choice



print(Question.objects.get(pk=1))

1. 좋아하는 동물은 무엇입니까?


In [ ]:
# Model클래스.objects : Model Manager 
#        - Model클래스와 연결된 테이블에 SQL(select)문을 실행하는 메소드를 제공
# 
type(Question.objects), type(Choice.objects)

(django.db.models.manager.Manager, django.db.models.manager.Manager)

In [ ]:
result = Question.objects.all()  # select * from question;
type(result) # 조회결과들을 담는 타입(QuerySet). 

django.db.models.query.QuerySet

In [ ]:
result[0] # subscriptable 타입

<Question: 1. 좋아하는 동물은 무엇입니까?>

In [ ]:
for q in result:  # Iterable
    print(q)

1. 좋아하는 동물은 무엇입니까?
2. 좋아하는 색은 무엇인가요?


In [8]:
print(result.query)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


# 조회
- all() : where절 없이 전체 행을 조회.
- where 절 조건으로 조회
  - filter()
  - exclude()
  - get()

In [9]:
from polls.models import Question, Choice #  모델 클래스들 import

# 전체 조회
result = Question.objects.all()
for q in result:
    print(q, type(q))

1. 좋아하는 동물은 무엇입니까? <class 'polls.models.Question'>
2. 좋아하는 색은 무엇인가요? <class 'polls.models.Question'>
3. 여행가고 싶은 나라는 어디인가요? <class 'polls.models.Question'>
4. 좋아하는 음식 종류를 선택하세요. <class 'polls.models.Question'>


In [ ]:
for q in result:
    print(q.id, q.pk, q.question_text, q.pub_date, sep=", ")
# 모델의 primary key Field의  값을 조회할 때: 모델.pk, 모델.field명

1, 1, 좋아하는 동물은 무엇입니까?, 2026-01-15 02:36:08.630013+00:00
2, 2, 좋아하는 색은 무엇인가요?, 2026-01-15 02:36:23.201466+00:00
3, 3, 여행가고 싶은 나라는 어디인가요?, 2026-01-15 03:49:46.402984+00:00
4, 4, 좋아하는 음식 종류를 선택하세요., 2026-01-15 03:51:14.389219+00:00


In [ ]:
# indexing
result[0]
# result[-1] # 음수 index는 지원안함.

ValueError: Negative indexing is not supported.

In [15]:
result[2:]

[<Question: 3. 여행가고 싶은 나라는 어디인가요?>, <Question: 4. 좋아하는 음식 종류를 선택하세요.>]

In [17]:
result.first()

<Question: 1. 좋아하는 동물은 무엇입니까?>

In [18]:
result.last()

<Question: 4. 좋아하는 음식 종류를 선택하세요.>

In [ ]:
# result = Question.objects.get(id=1) #id: field명 
result = Question.objects.get(pk=1)   # primary key Field = 1
print(type(result))
result.id, result.question_text, result.pub_date

<class 'polls.models.Question'>


(1,
 '좋아하는 동물은 무엇입니까?',
 datetime.datetime(2026, 1, 15, 2, 36, 8, 630013, tzinfo=datetime.timezone.utc))

In [22]:
try:
    Question.objects.get(id=10) # id=10 =>  where id = 10
except:
    print("조회결과가 없습니다.")

조회결과가 없습니다.


In [26]:
# try:
Choice.objects.get(votes=0) # where votes = 0
# except:
#     print("조회결과가 1개 이상입니다.")

MultipleObjectsReturned: get() returned more than one Choice -- it returned 3!

In [ ]:
result = Choice.objects.filter(votes=0) # where votes = 0
for c in result:
    print(c.id, c.pk, c.choice_text, c.votes, c.question_id, c.question)
    # c.question : 참조 모델 객체, c.question_id (모델이름_pk이름): 참조 모델의 pk값

2 2 호랑이 0 1 1. 좋아하는 동물은 무엇입니까?
3 3 녹색 0 2 2. 좋아하는 색은 무엇인가요?
5 5 빨강색 0 2 2. 좋아하는 색은 무엇인가요?


In [32]:
result = Question.objects.exclude(pk=1)
for r in result:
    print(r)

2. 좋아하는 색은 무엇인가요?
3. 여행가고 싶은 나라는 어디인가요?
4. 좋아하는 음식 종류를 선택하세요.


In [40]:
############################
# field 조건
# field이름__연산자 = 비교값
############################
result = Choice.objects.filter(votes=0) # where votes = 0
result = Choice.objects.filter(votes__lt=20)  # where votes < 20
result = Choice.objects.filter(votes__lte=20)  # where votes <= 20
result = Choice.objects.filter(votes__gt=20)  # where votes > 20
result = Choice.objects.filter(votes__gte=20)  # where votes >= 20
result = Choice.objects.filter(choice_text='빨강색')

for q in result:
    print(q.id, q.choice_text, q.votes, sep=", ")

print(result.query)

5, 빨강색, 0
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."choice_text" = 빨강색


In [ ]:
# 문자열 부분일치 조회

# where question_text like '좋아하는%'
result = Question.objects.filter(question_text__startswith="좋아하는")

# where not question_text like '좋아하는%'
result = Question.objects.exclude(question_text__startswith="좋아하는")

# where not question_text like '%무엇인가요?'
result = Question.objects.filter(question_text__endswith="무엇인가요?")

# where not question_text like '%동물%'
result = Question.objects.filter(question_text__contains="동물")

for q in result:
    print(q.id, q.question_text, q.pub_date.strftime('%Y-%m-%d'), sep=", ")

print(result.query)

1, 좋아하는 동물은 무엇입니까?, 2026-01-15
6, 싫어 하는 동물은 무엇인가요?, 2026-01-15
SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE "polls_question"."question_text" LIKE %동물% ESCAPE '\'


In [ ]:
# IN, between
# where id in [1, 3, 5]
result = Choice.objects.filter(pk__in=[1, 3, 5, 10])

# where votes between 10 and 30
result = Choice.objects.filter(votes__range=[10, 30])

for c in result:
    print(c, c.votes)

print(result.query)

1. 사자 5
3. 녹색 0
5. 빨강색 0
10. 미국 17
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."id" IN (1, 3, 5, 10)


In [56]:
# 조건이 여러개인 경우 - AND 
# where id >= 5 and votes < 20
result = Choice.objects.filter(
    pk__gte=5,
    votes__lt=20
)

for c in result:
    print(c, c.votes)

print(result.query)

5. 빨강색 0
6. 보라색 10
8. 고양이 5
10. 미국 17
11. 중국 17
13. 영국 8
14. 독일 6
19. 빨강색 3
20. 검정색 12
21. 파랑색 18
23. 하이에나 2
24. 뱀 11
26. 곰 9
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE ("polls_choice"."id" >= 5 AND "polls_choice"."votes" < 20)


In [ ]:
# OR - Q객체에 조건들을 넣어준다. Q(조건) | Q(조건) |
from django.db.models import Q

# where votes >= 20 or id <= 10
result = Choice.objects.filter(
    Q(votes__gte=20) | Q(pk__lte=10)
)

# where not (votes >= 20) or id <= 10
result = Choice.objects.filter(
    ~Q(votes__gte=20) | Q(pk__lte=10)
)

for c in result:
    print(c, c.votes)

print(result.query)

1. 사자 5
2. 호랑이 0
3. 녹색 0
4. 파랑색 20
5. 빨강색 0
6. 보라색 10
7. 개 50
8. 고양이 5
9. 팬더 23
10. 미국 17
11. 중국 17
13. 영국 8
14. 독일 6
19. 빨강색 3
20. 검정색 12
21. 파랑색 18
23. 하이에나 2
24. 뱀 11
26. 곰 9
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE (NOT ("polls_choice"."votes" >= 20) OR "polls_choice"."id" <= 10)


In [63]:
# 정렬 - QuerySet.order_by("컬럼명", "컬럼명")  "-컬럼명": 내림차순(desc)
# order by votes asc id desc
result = Choice.objects.all().order_by("votes", "-id")

for c in result:
    print(c, c.votes)

print(result.query)

5. 빨강색 0
3. 녹색 0
2. 호랑이 0
23. 하이에나 2
19. 빨강색 3
8. 고양이 5
1. 사자 5
14. 독일 6
13. 영국 8
26. 곰 9
6. 보라색 10
24. 뱀 11
20. 검정색 12
11. 중국 17
10. 미국 17
21. 파랑색 18
4. 파랑색 20
25. 호랑이 23
9. 팬더 23
18. 일식 26
12. 프랑스 27
22. 갈색 32
16. 중식 32
7. 개 50
17. 양식 67
15. 한식 79
SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."votes" ASC, "polls_choice"."id" DESC


In [67]:
# 특정 컬럼들만 조회
result = Choice.objects.all().values("pk", "choice_text")
print(type(result))

for c in result:
    print(type(c), c, c['choice_text'])  # 개별 조회결과를 dictionary로 반환.

<class 'django.db.models.query.QuerySet'>
<class 'dict'> {'pk': 1, 'choice_text': '사자'} 사자
<class 'dict'> {'pk': 2, 'choice_text': '호랑이'} 호랑이
<class 'dict'> {'pk': 3, 'choice_text': '녹색'} 녹색
<class 'dict'> {'pk': 4, 'choice_text': '파랑색'} 파랑색
<class 'dict'> {'pk': 5, 'choice_text': '빨강색'} 빨강색
<class 'dict'> {'pk': 6, 'choice_text': '보라색'} 보라색
<class 'dict'> {'pk': 7, 'choice_text': '개'} 개
<class 'dict'> {'pk': 8, 'choice_text': '고양이'} 고양이
<class 'dict'> {'pk': 9, 'choice_text': '팬더'} 팬더
<class 'dict'> {'pk': 10, 'choice_text': '미국'} 미국
<class 'dict'> {'pk': 11, 'choice_text': '중국'} 중국
<class 'dict'> {'pk': 12, 'choice_text': '프랑스'} 프랑스
<class 'dict'> {'pk': 13, 'choice_text': '영국'} 영국
<class 'dict'> {'pk': 14, 'choice_text': '독일'} 독일
<class 'dict'> {'pk': 15, 'choice_text': '한식'} 한식
<class 'dict'> {'pk': 16, 'choice_text': '중식'} 중식
<class 'dict'> {'pk': 17, 'choice_text': '양식'} 양식
<class 'dict'> {'pk': 18, 'choice_text': '일식'} 일식
<class 'dict'> {'pk': 19, 'choice_text': '빨강색'} 빨강색
<clas

# 집계

- aggregate(집계함수객체(컬럼명), 집계함수객체(컬럼명), ...) 
  - 전체행 기준 집계
- values('groupby 기준 컬럼').annotate(집계함수객체(컬럼명), ..) 
  -  Group by 후 집계

In [ ]:
from django.db import models
# select count(id), avg(votes) from choice;
result = Choice.objects.aggregate(
    models.Count("id"),
    models.Avg("votes")
)
# 결과 key를 지정
# select count(id) as cnt, avg(votes) as avg from choice;
result = Choice.objects.aggregate(
    cnt = models.Count("id"),
    avg = models.Avg("votes")
)

result

{'cnt': 26, 'avg': 19.307692307692307}

In [74]:
result['cnt']

26

In [76]:
# Group by 집계
# select   min(vote), max(vote)    from choice groupby question
result = Choice.objects.values("question").annotate(
    models.Min('votes'), models.Max('votes')
)

print(type(result))
for r in result:
    print(r)

<class 'django.db.models.query.QuerySet'>
{'question': 1, 'votes__min': 0, 'votes__max': 50}
{'question': 2, 'votes__min': 0, 'votes__max': 20}
{'question': 3, 'votes__min': 6, 'votes__max': 27}
{'question': 4, 'votes__min': 26, 'votes__max': 79}
{'question': 5, 'votes__min': 3, 'votes__max': 32}
{'question': 6, 'votes__min': 2, 'votes__max': 23}


# JOIN

In [ ]:
# 자식 모델 조회
# 자식 모델 기준으로 참조 부모 객체를 조회 -> ForeignKey 속성으로 조회.

choice = Choice.objects.get(pk=1) # Model 반환.
# Choice.question 필드 -> ForeignKey 필드 (Question참조)
question = choice.question
print(type(question))
question

<class 'polls.models.Question'>


<Question: 1. 좋아하는 동물은 무엇입니까?>

In [82]:
result = Choice.objects.filter(choice_text__contains='색')
for c in result:
    print(c.choice_text, c.question.question_text, sep=", ")

녹색, 좋아하는 색은 무엇인가요?
파랑색, 좋아하는 색은 무엇인가요?
빨강색, 좋아하는 색은 무엇인가요?
보라색, 좋아하는 색은 무엇인가요?
빨강색, 당신이 싫어 하는 색은 무엇입니까?
검정색, 당신이 싫어 하는 색은 무엇입니까?
파랑색, 당신이 싫어 하는 색은 무엇입니까?
갈색, 당신이 싫어 하는 색은 무엇입니까?


In [ ]:
# 부모테이블 조회결과를 이용해서 자식 테이블 참조 값들을 조회
## 부모Model객체.related_name : 
#            부모Model객체를 참조하는 자식의 행들로 부터 조회할 수 있는 ModelManager가 반환됨
#
# related_name: 자식모델클래스이름(소문자)_set (default이름) 
#               ForeignKey(..., related_name="relatedName을 지정")

question = Question.objects.get(pk=1)
choice_list = question.choice_set.all() # all 대상: question(pk=1)을 참조하는 choice 조회대상.

print("질문:", question.question_text)
for idx, ch in enumerate(choice_list, start=1):
    print(idx, ch.choice_text)

# type(question.choice_set)

질문: 좋아하는 동물은 무엇입니까?
1 사자
2 호랑이
3 개
4 고양이
5 팬더


# Insert/Update

In [91]:
# 모델 객체를 생성 - Initializer에 keyword 인자로 Field의 값을 대입.
new_question = Question(question_text="새로운 질문입니다.")
print(new_question.pk, new_question.question_text)
new_question.save() # insert

None 새로운 질문입니다.


In [96]:
q = Question.objects.get(pk=2)
q.question_text = "당신이 좋아하는 색깔은 뭔가요?"
print(q.pk, q.question_text)
q.save()  # pk=2 인 행을 update

2 당신이 좋아하는 색깔은 뭔가요?


In [97]:
result = Question.objects.all()
for q in result:
    print(q)

1. 좋아하는 동물은 무엇입니까?
2. 당신이 좋아하는 색깔은 뭔가요?
3. 당신이 여행 가고 싶은 나라는 어디인가요?
4. 좋아하는 음식 종류를 선택하세요.
5. 당신이 싫어 하는 색은 무엇입니까?
6. 싫어 하는 동물은 무엇인가요?
7. 새로운 질문입니다.


In [98]:
#  삭제
del_q = Question(pk=7)
del_q.delete()  # 모델의 pk값을 삭제

(1, {'polls.Question': 1})

In [99]:
result = Question.objects.all()
for q in result:
    print(q)

1. 좋아하는 동물은 무엇입니까?
2. 당신이 좋아하는 색깔은 뭔가요?
3. 당신이 여행 가고 싶은 나라는 어디인가요?
4. 좋아하는 음식 종류를 선택하세요.
5. 당신이 싫어 하는 색은 무엇입니까?
6. 싫어 하는 동물은 무엇인가요?


In [102]:
# 삭제 전에 개수
cnt1 = Choice.objects.aggregate(models.Count("id"))
cnt1

{'id__count': 26}

In [103]:
# delete from tb where 삭제 조건 -> filter로 삭제할 조건의 모델을 조회 후에 삭제
# delete from choice where votes < 10

del_list = Choice.objects.filter(votes__lt=10)
for c in del_list:
    # print(c)
    c.delete()

In [104]:
cnt2 = Choice.objects.aggregate(models.Count("id"))
cnt2

{'id__count': 16}